In [13]:
import requests
import pandas as pd
import xmltodict
import schedule
import time
from datetime import datetime, timedelta

In [14]:
places = pd.read_csv('서울시 주요 50장소 목록.csv', encoding='cp949')

In [ ]:
# AREA_NM : 장소명
# LIVE_PPLTN_STTS : 실시간 인구현황
# ROAD_TRAFFIC_STTS : 도로소통현황
# PRK_STTS : 주차장현황
# SUB_STTS : 지하철 실시간 도착 현황
# BUS_STN_STTS : 버스정류소 현황
# ACDNT_CNTRL_STTS : 사고통제현황
# SBIKE_STTS : 따릉이현황
# WEATHER_STTS : 날씨현황
# COVID_19_STTS : 코로나19현황
# AREA_CONGEST_LVL : 장소 혼잡도 지표
# AREA_CONGEST_MSG : 장소 혼잡도 지표 관련 메세지
# AREA_PPLTN_MIN : 실시간 인구 지표 최소값
# AREA_PPLTN_MAX : 실시간 인구 지표 최대값
# 성별_PPLTN_RATE : 성별 인구 비율
# PPLTN_RATE_나이대 : 나이대 인구 비율
# RESNT_PPLTN_RATE : 상주 인구 비율
# NON_RESNT_PPLTN_RATE : 비상주 인구 비율
# REPLACE_YN : 대체 데이터 여부
# PPLTN_TIME : 실시간 인구 데이터 업데이트 시간

In [102]:
place_df = pd.DataFrame(columns=['수집일시', '장소', '최대인구', '최소인구', '10대', '2030대', '4050대', '6070대'])

In [100]:
def placeData():
    for place in places.장소명:
        url = f'http://openapi.seoul.go.kr:8088/7674687567666b6437377848736968/xml/citydata/1/1/{place}'
        response = requests.get(url)
        place_info = xmltodict.parse(response.text)

        try:
            place_name = place_info['SeoulRtd.citydata']['CITYDATA']['AREA_NM']
            place_people = place_info['SeoulRtd.citydata']['CITYDATA']['LIVE_PPLTN_STTS']['LIVE_PPLTN_STTS']

        except:
            pass

        else: 
            df_info = [
                datetime.now().strftime('%Y-%m-%d %H'),
                place_name,
                place_people['AREA_PPLTN_MAX'],
                place_people['AREA_PPLTN_MIN'],
                pd.to_numeric(place_people['PPLTN_RATE_10'])+pd.to_numeric(place_people['PPLTN_RATE_0']),
                pd.to_numeric(place_people['PPLTN_RATE_20'])+pd.to_numeric(place_people['PPLTN_RATE_30']),
                pd.to_numeric(place_people['PPLTN_RATE_40'])+pd.to_numeric(place_people['PPLTN_RATE_50']),
                pd.to_numeric(place_people['PPLTN_RATE_60'])+pd.to_numeric(place_people['PPLTN_RATE_70'])
            ]
            place_df.loc[place_df.shape[0]] = df_info
            
    print('추가 완료', end='\n\n')
    
schedule.every(1).minutes.do(placeData)
endDate = (datetime.now()+timedelta(days=4)).strftime('%Y-%m-%d')

while True:
    schedule.run_pending()
    if endDate == datetime.now().strftime('%Y-%m-%d'):
        schedule.clear()
        break;
place_df.to_csv('장소데이터.csv', en)

추가 완료



KeyboardInterrupt: 

In [103]:
schedule.clear()
schedule.jobs

[]

In [101]:
place_df

,수집일시,장소,최대인구,최소인구,10대,2030대,4050대,6070대
0,2023-05-19 19,강남 MICE 관광특구,18000,16000,5.3,55.8,33.0,5.8
1,2023-05-19 19,동대문 관광특구,26000,24000,5.3,50.3,33.9,10.5
2,2023-05-19 19,명동 관광특구,56000,54000,3.2,52.1,36.6,8.1
3,2023-05-19 19,이태원 관광특구,12000,10000,4.5,61.5,25.2,8.8
4,2023-05-19 19,잠실 관광특구,68000,66000,14.1,54.0,23.7,8.2
...,...,...,...,...,...,...,...,...
95,2023-05-19 19,서울숲공원,16000,14000,5.1,52.8,31.7,10.4
96,2023-05-19 19,월드컵공원,7000,6500,4.8,39.0,41.5,14.8
97,2023-05-19 19,이촌한강공원,5000,4500,4.2,41.4,38.1,16.3
98,2023-05-19 19,잠실종합운동장,10000,9500,13.6,55.2,26.8,4.5
